In [1]:
import time
import datetime

import cst_python as cst
import toolpy as tp
from toolpy.integrations import groq

import gbyg as gg

In [2]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [3]:
importance_threshould = 1

In [4]:
mind = cst.Mind()

gg.agent.agent_constructor(mind, importance_threshould)

for codelet in mind.code_rack.all_codelets:
    codelet.time_step = 100

memories_input = mind.raw_memory.get_all_of_type("MemoriesInput")[0]
memory_stream = mind.raw_memory.get_all_of_type("MemoryStream")[0]
agent_time = mind.raw_memory.get_all_of_type("AgentTime")[0]
agent_info = mind.raw_memory.get_all_of_type("AgentInfo")[0]
agent_summary_description = mind.raw_memory.get_all_of_type("AgentSummaryDescription")[0]
actual_place = mind.raw_memory.get_all_of_type("ActualPlace")[0]
previous_day_summary = mind.raw_memory.get_all_of_type("PreviousDaySummary")[0]
plan = mind.raw_memory.get_all_of_type("Plan")[0]

In [5]:
agent_info.set_info({"name":"Alex", "age":25, "traits":"friendly, outgoing, hospitable"})
agent_time.set_info(datetime.datetime.fromisoformat("2024-06-26T00:00:00").timestamp())
actual_place.set_info("Bedroom")

-1

In [6]:
mind.start()

In [7]:
input_memories  = [
  {
    "index": 0,
    "type": "event",
    "created": "2024-06-01 08:00:00",
    "description": "Alex attended a nature photography workshop in the city park.",
  },
  {
    "index": 1,
    "type": "reflection",
    "created": "2024-06-01 19:00:00",
    "description": "Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.",
  },
  {
    "index": 2,
    "type": "event",
    "created": "2024-06-05 10:00:00",
    "description": "Alex completed a major software development project at work, delivering the final product to the client.",
  },
  {
    "index": 3,
    "type": "reflection",
    "created": "2024-06-05 20:00:00",
    "description": "After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.",
  },
  {
    "index": 4,
    "type": "event",
    "created": "2024-06-10 16:00:00",
    "description": "Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.",
  }
]

for m in input_memories:
    m["created"] = datetime.datetime.fromisoformat(m["created"]).timestamp()
    m["last_acessed"] = m["created"]

In [8]:
input_memories

[{'index': 0,
  'type': 'event',
  'created': 1717239600.0,
  'description': 'Alex attended a nature photography workshop in the city park.',
  'last_acessed': 1717239600.0},
 {'index': 1,
  'type': 'reflection',
  'created': 1717279200.0,
  'description': 'Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.',
  'last_acessed': 1717279200.0},
 {'index': 2,
  'type': 'event',
  'created': 1717592400.0,
  'description': 'Alex completed a major software development project at work, delivering the final product to the client.',
  'last_acessed': 1717592400.0},
 {'index': 3,
  'type': 'reflection',
  'created': 1717628400.0,
  'description': 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
  'last_acessed': 1717628400.0},
 {'index': 4,
  'type': 'event',
  'created': 1718046000.0,
  'description': 'Alex went on a weekend trip to explore a national park, taking 

In [9]:
memories_input_queue = memories_input.get_info()
for memory in input_memories:
    memories_input_queue.append(memory)

In [10]:
time.sleep(10)

print(len(memory_stream.get_info()))

D:\Github\GDataByGAgents\src\gbyg\agent\retrieval\recency_scorer_tool.py:32: RuntimeWarning: invalid value encountered in divide
  scores = (scores-min_score)/(max_score-min_score)
c:\Python38\lib\site-packages\sentence_transformers\util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  b = torch.tensor(b)


10


In [11]:
for memory in memory_stream.get_info():
    because_of = ""
    if 'because_of' in memory: 
        because_of = memory['because_of']

    memory_str = f"{memory['index']} ({memory['type']}): {memory['description']} | because of: {because_of}"
    print(memory_str)

0 (event): Alex attended a nature photography workshop in the city park. | because of: 
1 (reflection): Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects. | because of: 
2 (event): Alex completed a major software development project at work, delivering the final product to the client. | because of: 
3 (reflection): After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome. | because of: 
4 (event): Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes. | because of: 
5 (reflection): Alex is interested in photography | because of: [0, 4]
6 (reflection): Alex is capable of completing complex projects | because of: [2, 3]
7 (reflection): Alex values personal and professional development | because of: [0, 2]
8 (reflection): Alex enjoys exploring nature and taking photos | because of: [0, 4]
9 (reflection): Alex is reflective 

In [12]:
print(agent_summary_description.get_info())

{'summary': 'Name: Alex (age: 25)\nInnate traits: friendly, outgoing, hospitable\nAlex is an outdoorsy and creative person interested in photography.\nAlex is possibly a nature photographer or has an interest in photography.\nAlex seems enthusiastic and engaged.', 'generated_time': 1719370800.0}


In [13]:
agent_time.set_info(agent_time.get_info()+gg.time_utils.TimeInterval.ONE_DAY)

previous_day_summary_content = {"summary":'''Yesteday, 26 June, Alex went to the park, where he participated in a photography workshop. 
Alex took several photos, preparing to later edit them and post them on social media.
Alex now plans to stay home the entire day.''',
                        "generated_time":agent_time.get_info()}

previous_day_summary.set_info(previous_day_summary_content)

time.sleep(5)
print(agent_summary_description.get_info()["summary"])

Name: Alex (age: 25)
Innate traits: friendly, outgoing, hospitable
Alex is an contemplative and creative individual with a strong connection to the outdoors
Nature photographer
Alex has a contemplative and appreciative mindset, taking time to reflect on their life's journey, and finds joy in capturing the beauty of nature through photography.


In [14]:
time.sleep(10)

In [15]:
for action in plan.get_info():
    hour = action["hour"]
    minute = action["minute"]
    action_action = action["action"]

    print(f"{hour}:{minute}: {action_action}")

8:0: Wakes up
8:10: Has breakfast
9:0: Get out photographs from yesterday's photography workshop
9:15: Organize photographs
12:0: Takes a lunch break
12:10: Scrolls through social media
13:0: Resume editing photos
18:0: Start getting ready for dinner
18:10: Take a short break to eat dinner
18:30: Finish dinner
19:0: Continue editing photos
19:20: Finalize photo edits
19:30: Prepare photos for online posting
21:0: Edit photos
21:30: Relax at home
22:0: Choose what to watch
22:10: Start watching
22:50: Get ready for bed
23:0: Goes to sleep


In [16]:
raise ValueError()

ValueError: 

In [ ]:
print(len(memory_stream.get_info()))

10


In [ ]:
alex_house = {
    "Alex's House": {
        "Living Room": [
            {"name": "sofa", 'type':'inanimate_object', 
             "description": "A comfortable, modern sofa perfect for relaxing or hosting friends.",
             "state":"idle and clean"},
            {"name": "coffee table", 'type':'inanimate_object', 
             "description": "A sleek coffee table with a few programming books and the latest tech magazines.",
             "state":"idle and clean"},
            {"name": "television", 'type':'inanimate_object', 
             "description": "A large smart TV, often used to watch tech talks or unwind with a show.",
             "state":"off and clean"},
            {"name": "bookshelf", 'type':'inanimate_object', 
             "description": "A bookshelf filled with a mix of classic literature and technical manuals.",
             "state":"idle and clean"},
            {"name": "rug", 'type':'inanimate_object', 
             "description": "A soft rug that adds a touch of warmth to the living room.",
             "state":"idle and clean"}
        ],
        "Kitchen": [
            {"name": "refrigerator", 'type':'inanimate_object', 
             "description": "A high-tech refrigerator that keeps Alex's meals and snacks fresh.",
             "state":"on and clean"},
            {"name": "stove", 'type':'inanimate_object', 
             "description": "A modern stove where Alex occasionally experiments with new recipes.",
             "state":"off and clean"},
            {"name": "microwave", 'type':'inanimate_object', 
             "description": "A quick and efficient microwave for heating up meals.",
             "state":"off and clean"},
            {"name": "kitchen sink", 'type':'inanimate_object', 
             "description": "A stainless steel sink",
             "state":"idle and clean"},
            {"name": "dishwasher", 'type':'inanimate_object', 
             "description": "A quiet dishwasher that handles the post-dinner cleanup.",
             "state":"off and clean"},
            {"name": "dining table", 'type':'inanimate_object', 
             "description": "A minimalist dining table where Alex enjoys breakfast and dinner.",
             "state":"idle and clean"},
            {"name": "chairs", 'type':'inanimate_object', 
             "description": "Comfortable chairs around the dining table, perfect for meals or working on a laptop.",
             "state":"idle and clean"}
        ],
        "Bedroom": [
            {"name": "bed", 'type':'inanimate_object', 
             "description": "A cozy bed where Alex gets his much-needed rest.",
             "state":"idle and clean"},
            {"name": "dresser", 'type':'inanimate_object', 
             "description": "A dresser with neatly organized clothes, reflecting Alex's organized nature.",
             "state":"idle and clean"},
            {"name": "nightstand", 'type':'inanimate_object', 
             "description": "A nightstand with a lamp and a few personal items.",
             "state":"idle and clean"},
            {"name": "closet", 'type':'inanimate_object', 
             "description": "A spacious closet with casual and work clothes.",
             "state":"idle and clean"},
            {"name": "desk", 'type':'inanimate_object', 
             "description": "A small desk with a notebook and pens, used for jotting down late-night ideas.",
             "state":"idle and clean"},
            {"name": "lamp", 'type':'inanimate_object', 
             "description": "A bedside lamp for reading before bed.",
             "state":"off and clean"}
        ],
        "Bathroom": [
            {"name": "shower", 'type':'inanimate_object', 
             "description": "A modern shower with various settings for a refreshing start to the day.",
             "state":"off and clean"},
            {"name": "bathroom sink", 'type':'inanimate_object', 
             "description": "A clean sink with essential toiletries.",
             "state":"idle and clean"},
            {"name": "toilet", 'type':'inanimate_object', 
             "description": "A simple, modern toilet.",
             "state":"idle and clean"},
            {"name": "mirror", 'type':'inanimate_object', 
             "description": "A mirror above the sink, used for daily grooming.",
             "state":"idle and clean"},
            {"name": "cabinet", 'type':'inanimate_object', 
             "description": "A cabinet filled with towels and personal care items.",
             "state":"idle and clean"}
        ],
        "Home Office": [
            {"name": "desk", 'type':'inanimate_object', 
             "description": "A large desk with multiple monitors and a laptop, where Alex spends most of his working hours.",
             "state":"idle and clean"},
            {"name": "computer", 'type':'inanimate_object', 
             "description": "A powerful computer equipped with the latest software development tools.",
             "state":"off and clean"},
            {"name": "office chair", 'type':'inanimate_object', 
             "description": "An ergonomic office chair to support long coding sessions.",
             "state":"idle and clean"},
            {"name": "bookshelf", 'type':'inanimate_object', 
             "description": "A bookshelf filled with programming books and reference materials.",
             "state":"idle and clean"},
            {"name": "printer", 'type':'inanimate_object',
              "description": "A reliable printer for printing documents and designs.",
              "state":"off and clean"}
        ]
    }
}

alex = {"name":"Alex", "type":"agent", "description":"human", 'state':'idle, standing in the middle of the room.'}

alex_house["Alex's House"]["Bedroom"].append(alex)

agent_names = ["Alex"]

In [ ]:
observation_memory = {"index": 0,
    "type": "event",
    "created": agent_time.get_info(),
    "description": f"Alex is {alex['state']}"}

input_memories.append(observation_memory)

In [ ]:
env = gg.environment.Environment(alex_house, agent_names)

In [ ]:
actions = ["turn on the lamp", "go to the kitchen"]

for step, action in enumerate(actions):
    agent_actions = {"Alex":action}

    obsevation = env.step(agent_actions)
    alex_observation = obsevation["Alex"]

    observation_memory = {"type": "event",
        "created": agent_time.get_info(),
        "description": f"Alex is {alex_observation['state']}"}

    memories_input_queue.append(observation_memory)

    observation_memory = {"type": "event",
        "created": agent_time.get_info(),
        "description": alex_observation['observation']}
    
    memories_input_queue.append(observation_memory)

    actual_place.set_info(alex_observation["place"])

    current_time = agent_time.get_info()
    current_time += alex_observation["duration"]
    agent_time.set_info(current_time)

In [ ]:
time.sleep(10)
print(len(memory_stream.get_info()))

14


In [ ]:
for i in range(len(memory_stream.get_info())-4, len(memory_stream.get_info())):
    m = memory_stream.get_info()[i]
    print(m["importance"],"|", m["description"])

0.1111111111111111 | Alex is idle, standing in the middle of the room.
0.3888888888888889 | Alex sees the cozy bed, dresser, nightstand, closet, and desk in the bedroom. Alex sees himself standing in the middle of the room. Alex hears nothing but silence. Alex feels the soft carpet under his feet. Alex notices the lamp is now on, illuminating the room.
0.0 | Alex is walking
0.16666666666666666 | Alex sees a modern stove, refrigerator, microwave, kitchen sink, dishwasher, and dining table in the kitchen. He hears the hum of the refrigerator and the faint sound of the dishwasher. He feels the tile floor beneath his feet and the cool air from the air conditioner on his skin.
